In [2]:
from filterpy.kalman import KalmanFilter
from simglucose.simulation.user_interface import simulate
from simglucose.simulation.env import T1DSimEnv
from simglucose.controller.basal_bolus_ctrller import BBController
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.simulation.scenario import CustomScenario
from simglucose.simulation.sim_engine import SimObj, sim, batch_sim
from datetime import timedelta
from datetime import datetime
import ipywidgets as widgets
import random
from IPython.display import display
import numpy as np


# ----------Simulation
# Code adapted from https://github.com/jxx123/simglucose

# Random Scenario
now = datetime.now()
start_time = datetime.combine(now.date(), datetime.min.time())
# Specify results saving path
path = './results'
# Create a simulation environment
patient = T1DPatient.withName('adolescent#003')
sensor = CGMSensor.withName('Dexcom', seed=1)
pump = InsulinPump.withName('Insulet')
scenario = RandomScenario(start_time=start_time, seed=random.randrange(1,2000))
env = T1DSimEnv(patient, sensor, pump, scenario)
# Create a controller
controller = BBController()

# Put them together to create a simulation object
s1 = SimObj(env, controller, timedelta(days=1), animate=False, path=path)
results1 = sim(s1)
print(type(results1))
actual = results1.loc[:,"BG"]
observation = results1.loc[:,"CGM"]


f = KalmanFilter (dim_x=1, dim_z=1)

#Initial Value
f.x = np.array([1])   # velocity

#State Transition Matrix
f.F = np.array([1])

#Measurement Function
f.H = np.array([1]) 

# Transition Covariance
f.P *= 0.1

# Measurement Noise
f.R = 5

n_timesteps = 481
x = np.linspace(0, 3 * np.pi, n_timesteps)


f.Q = 0.0001    
for t in range(n_timesteps-1):
    #z = get_sensor_reading()
    print("obs", observation[t])
    f.update([observation[t]])
    #f.update(z)
    print(f.x)





Process ID: 1996
Simulation starts ...
Simulation Completed!
<class 'pandas.core.frame.DataFrame'>
obs 162.6739493687905


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional